<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_Sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install kaggle

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!kaggle datasets download -d nehasontakke/amazon-unlocked-mobilecsv --unzip


Dataset URL: https://www.kaggle.com/datasets/nehasontakke/amazon-unlocked-mobilecsv
License(s): CC0-1.0


In [7]:
import pandas as pd

df = pd.read_csv("Amazon_Unlocked_Mobile.csv")

# Show first few rows
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [8]:
df=df.drop_duplicates()

In [9]:
# Drop missing values
df.dropna(inplace=True)
df.drop(columns=['Product Name',	'Brand Name', 'Price','Review Votes'], inplace=True)
# Check basic info
(df.describe())

,Rating
count,281249.000000
mean,3.785781
std,1.553934
min,1.000000
25%,3.000000
50%,5.000000
75%,5.000000
max,5.000000


In [10]:
# Check basic info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 281249 entries, 0 to 413829
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Rating   281249 non-null  int64 
 1   Reviews  281249 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.4+ MB
None


In [11]:
import numpy as np
# Remove any 'neutral' ratings equal to 3
df = df[df['Rating'] != 3]

# Encode 4s and 5s as 1 (rated positively)
# Encode 1s and 2s as 0 (rated poorly)
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df.head()

<ipython-input-11-d6c87b1a21b4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)


,Rating,Reviews,Positively Rated
0,5,I feel so LUCKY to have found this used (phone...,1
1,4,"nice phone, nice up grade from my pantach revu...",1
2,5,Very pleased,1
3,4,It works good but it goes slow sometimes but i...,1
4,4,Great phone to replace my lost phone. The only...,1


In [12]:
df.drop(columns=['Rating'], inplace=True)
df.head()

<ipython-input-12-7ac5f1f75c50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Rating'], inplace=True)


,Reviews,Positively Rated
0,I feel so LUCKY to have found this used (phone...,1
1,"nice phone, nice up grade from my pantach revu...",1
2,Very pleased,1
3,It works good but it goes slow sometimes but i...,1
4,Great phone to replace my lost phone. The only...,1


In [13]:
X=df['Reviews']
y=df['Positively Rated']

In [14]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simple & Lightweight: TF-IDF (Best for Small Datasets)

Use when: You have limited compute, small datasets (<100k reviews).

Cons: No contextual understanding of words.

In [15]:
#TFIDF or term frequency inverse document frequency allows us to weight terms based on how important they are to a document.
#High weight is given to terms that appear often in a particular document, but don't appear often in the corpus.
#Features with low TFIDF are either commonly used across all documents or are rarely used and only occur in long documents.
#Features with high TFIDF are frequently used within specific documents, but rarely used across all documents.

from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5 (Ignore words that appear in fewer than 5 documents.)
vect = TfidfVectorizer(min_df=5, max_features=10000).fit(X_train)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
X_train_vectorized = vect.transform(X_train)

# Logistic Regression with regularization
model = LogisticRegression(
    penalty='l2',           # L2 regularization (default)
    C=1.0,                  # Inverse of regularization strength; lower = stronger regularization
    max_iter=100
)

model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.9233086416177297


In [17]:
len(vect.get_feature_names_out())
feature_names = np.array(vect.get_feature_names_out())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['1300' '700nits' 'a10' 'resins' '___thank' 'glues' 'messiah' 'bigtime'
 'seizing' '16nm']

Largest tfidf: 
['completamente' 'medium' 'love' 'loved' 'loveit' 'lovely' 'didnot' 'didn'
 'did' 'device']


In [18]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not' 'worst' 'useless' 'return' 'waste' 'disappointed' 'horrible'
 'returning' 'terrible' 'poor']

Largest Coefs: 
['love' 'great' 'excellent' 'amazing' 'perfect' 'awesome' 'perfectly'
 'loves' 'easy' 'best']


In [19]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[0 0]


# n-grams with CountVectorizer: Fit the CountVectorizer to the training data specifiying a minimum document frequency of 5 and extracting 1-grams and 2-grams


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5, ngram_range=(1,2), max_features=10000).fit(X_train)

X_train_vectorized = vect.transform(X_train)

model = LogisticRegression(
    penalty='l2',           # L2 regularization (default)
    C=1.0,                  # Inverse of regularization strength; lower = stronger regularization
    max_iter=100
)

model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9369938379325575


In [21]:
feature_names = np.array(vect.get_feature_names_out())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not happy' 'not satisfied' 'junk' 'not worth' 'no good' 'garbage'
 'worst' 'not very' 'awful' 'not good']

Largest Coefs: 
['excelent' 'excelente' 'not bad' 'no problems' 'exelente' 'excellent'
 'perfecto' 'perfect' 'no issues' 'loves']


In [22]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5, ngram_range=(2,3), max_features=10000).fit(X_train)

X_train_vectorized = vect.transform(X_train)

model = LogisticRegression(
    penalty='l2',           # L2 regularization (default)
    C=1.0,                  # Inverse of regularization strength; lower = stronger regularization
    max_iter=100
)

model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8977930723204179


In [25]:
feature_names = np.array(vect.get_feature_names_out())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['never worked' 'very disappointed' 'not satisfied' 'dont buy'
 'very disappointing' 'is horrible' 'the worst' 'not recommend'
 'wouldn recommend' 'very bad']

Largest Coefs: 
['loves it' 'love this' 'love it' 'excellent phone' 'excellent product'
 'excelente producto' 'like is' 'was worried' 'amazing phone' 'all good']


In [26]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]


## Contextual Embeddings: BERT. Use when: You have a large dataset (>100k reviews) & GPU access.
## Pro: Captures context & sentiment much better than TF-IDF. Con: Computationally expensive.

In [2]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")


AttributeError: module 'numpy' has no attribute 'dtypes'

In [ ]:
embeddings = model.encode(X_train.astype(str).tolist())

# Print the shape of embeddings (Each review -> 384-dimensional vector)
print(embeddings.shape)  #

In [ ]:
test_embeddings = model.encode(X_test.astype(str).tolist())


In [ ]:
model = LogisticRegression(
    penalty='l2',           # L2 regularization (default)
    C=1.0,                  # Inverse of regularization strength; lower = stronger regularization
    max_iter=100
)

model.fit(embeddings, y_train)


In [ ]:
predictions = model.predict(test_embeddings)

print('AUC: ', roc_auc_score(y_test, predictions))